# **Attendance Data Consolidation and Transformation with Pandas**

**Note:** This notebook contains a modified version of codes I developed to improve an existing process during my previous employment. The techniques and data formats applied here are similar to those used in the real-world scenario, but the actual data, including names, numbers, and values, have been randomly generated for the purpose of illustration and confidentiality. The modifications aim to showcase the approach and techniques applied in the original context.

### **Background**

Code in this notebook was used to streamline the attendance reporting process that I used to perform on a daily basis. The daily task involved transforming schedule and phone data of over 500 service desk employees into a format that is uploaded to SQL for use in an attendance dashboard.

#### **Goal**

The goal is to generate two outputs from raw data in a quick and efficient manner.
1. Consolidated schedule data - Contains:
  * Date
  * Start time
  * End time  
  * Employee number
  * Code: 1 (Off), 2 (Shift), 3 (Training), 4 (Vacation), 5 (PTO)

2. Employee login logout data - Contains:
  * Employee name
  * Date
  * Login time
  * Logout time
  * Employee number

#### **Raw Data**

##### **Schedule data**

Schedule data is provided separately by each department. They all use the same Excel file template.

![picture](https://drive.google.com/uc?export=view&id=1OsR9ezvDlW5Pw3ORYCtt4g9LNmYaK_TD)

##### **Phone data**

Employee login/logout data is obtained from the software phone system in Excel format. A day may have multiple login and logout times

![picture](https://drive.google.com/uc?export=view&id=15MZvNlQopQJdMIvQ3qmwzsnTW-qINl8u)

##### **Employee List**

The employee list is also saved as an Excel file. It contains employee IDs, Z IDs and phone IDs.

![picture](https://drive.google.com/uc?export=view&id=1_as6A6QaJtbvSfLDHSRXP7j57ovyaJ-E)
